In [ ]:
from basketball_reference_scraper.shot_charts import get_shot_chart
from basketball_reference_scraper.pbp import get_pbp
from basketball_reference_scraper.teams import get_roster, get_team_stats, get_team_ratings
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
import pandas as pd
import numpy as np

In [1]:
from cryptography.fernet import Fernet

key = Fernet.generate_key()
print(key.decode())


Ag_7Fu0eFzy4i-IwPBloNKXN66FjNm2so_njB0fKksU=


In [18]:
nba_teams = {
    "Atlanta Hawks": "ATL",
    "Boston Celtics": "BOS",
    "Brooklyn Nets": "BRK",
    "Charlotte Hornets": "CHO",
    "Chicago Bulls": "CHI",
    "Cleveland Cavaliers": "CLE",
    "Dallas Mavericks": "DAL",
    "Denver Nuggets": "DEN",
    "Detroit Pistons": "DET",
    "Golden State Warriors": "GSW",
    "Houston Rockets": "HOU",
    "Indiana Pacers": "IND",
    "Los Angeles Clippers": "LAC",
    "Los Angeles Lakers": "LAL",
    "Memphis Grizzlies": "MEM",
    "Miami Heat": "MIA",
    "Milwaukee Bucks": "MIL",
    "Minnesota Timberwolves": "MIN",
    "New Orleans Pelicans": "NOP",
    "New York Knicks": "NYK",
    "Oklahoma City Thunder": "OKC",
    "Orlando Magic": "ORL",
    "Philadelphia 76ers": "PHI",
    "Phoenix Suns": "PHO",
    "Portland Trail Blazers": "POR",
    "Sacramento Kings": "SAC",
    "San Antonio Spurs": "SAS",
    "Toronto Raptors": "TOR",
    "Utah Jazz": "UTA",
    "Washington Wizards": "WAS"
}

nba_teams_simple = {
    "Hawks": "ATL",
    "Celtics": "BOS",
    "Nets": "BRK",
    "Hornets": "CHO",
    "Bulls": "CHI",
    "Cavaliers": "CLE",
    "Mavericks": "DAL",
    "Nuggets": "DEN",
    "Pistons": "DET",
    "Warriors": "GSW",
    "Rockets": "HOU",
    "Pacers": "IND",
    "Clippers": "LAC",
    "Lakers": "LAL",
    "Grizzlies": "MEM",
    "Heat": "MIA",
    "Bucks": "MIL",
    "Timberwolves": "MIN",
    "Pelicans": "NOP",
    "Knicks": "NYK",
    "Thunder": "OKC",
    "Magic": "ORL",
    "76ers": "PHI",
    "Suns": "PHO",
    "Trail Blazers": "POR",
    "Kings": "SAC",
    "Spurs": "SAS",
    "Raptors": "TOR",
    "Jazz": "UTA",
    "Wizards": "WAS"
}


In [ ]:
import requests
import pandas as pd
from datetime import datetime

url = "https://cdn.nba.com/static/json/staticData/scheduleLeagueV2.json"
res = requests.get(url)
data = res.json()

NOT_VALID_LABELS = {'Rising Stars Semifinal','Rising Stars Championship','All-Star Semifinal','All-Star Championship','Preseason'}
rows = []

for day in data['leagueSchedule']['gameDates']:
    for game in day['games']:
        if game['gameLabel'] not in NOT_VALID_LABELS:
            home = game['homeTeam']['teamName']
            away = game['awayTeam']['teamName']
            date = datetime.strptime(game['homeTeamTime'], "%Y-%m-%dT%H:%M:%SZ").date()
            arena = game.get('arenaName', '')
            city = game.get('arenaCity', '')
            state = game.get('arenaState', '')

            game_type = (
                "Regular Season" if game['gameLabel'] == ""
                else game['gameLabel']
            )

            rows.append({
                "home_team": home,
                "away_team": away,
                "date": date.strftime("%Y-%m-%d"),
                "arena": arena,
                "city": city,
                "state": state,
                "game_type": game_type
            })

df = pd.DataFrame(rows)

df["home_team"] = df["home_team"].map(nba_teams_simple)
df["away_team"] = df["away_team"].map(nba_teams_simple)
print(df.head())

def getDateTeam(df):
    a = df['date']
    b = df['home_team']
    c = df['away_team']
    return a, b, c

# Uzyskaj daty, drużyny gospodarza i gościa
date, home_team, away_team = getDateTeam(df)
#df.to_csv(r"C:\Users\tomai\Downloads\schedule.csv", index=False)

  home_team away_team        date                 arena         city state  \
0       BOS       NYK  2024-10-22             TD Garden       Boston    MA   
1       LAL       MIN  2024-10-22      Crypto.com Arena  Los Angeles    CA   
2       DET       IND  2024-10-23  Little Caesars Arena      Detroit    MI   
3       ATL       BRK  2024-10-23      State Farm Arena      Atlanta    GA   
4       MIA       ORL  2024-10-23         Kaseya Center        Miami    FL   

        game_type  
0  Regular Season  
1  Regular Season  
2  Regular Season  
3  Regular Season  
4  Regular Season  


In [ ]:
all_df = []

for i in range(len(df)):
    try:
        shots = get_shot_chart(date[i], home_team[i], away_team[i])
        for team_code, team_df in shots.items():
            team_df = team_df.copy()
            team_df["team"] = team_code
            team_df["date"] = date[i]
            all_df.append(team_df)
    except Exception as e:
        print(f"Error processing game on {date[i]} between {home_team[i]} and {away_team[i]}: {e}")


if all_df:
    final_df = pd.concat(all_df, ignore_index=True)
else:
    final_df = pd.DataFrame()

final_df.to_csv(r"C:\Users\tomai\Downloads\shot_charts.csv", index=False)
final_df


In [ ]:
all_df = pd.DataFrame()

for i in range(len(df)):
    try:
        pbp = get_pbp(date[i], home_team[i], away_team[i]) 
        pbp["date"] = date[i]
        pbp["team"] = np.where(
            pbp["HOME_ACTION"].isnull(),
            away_team[i],
            home_team[i]
        )
        all_df = pd.concat([all_df, pbp], ignore_index=True)
    except Exception as e:
        print(f"Error processing game on {date[i]} between {home_team[i]} and {away_team[i]}: {e}")

all_df.to_csv(r"C:\Users\tomai\Downloads\pbp.csv", index=False)
all_df


KeyboardInterrupt: 

In [19]:
all_df = pd.DataFrame()

for key, value in nba_teams.items():
    roster = get_roster(value, '2025')
    roster['team'] = value  
    all_df = pd.concat([all_df, roster], ignore_index=True)

all_df.to_csv(r"C:\Users\tomai\Downloads\roster_info.csv", index=False)
all_df.head()

Atlanta Hawks ATL
Boston Celtics BOS
Brooklyn Nets BRK
Charlotte Hornets CHO
Chicago Bulls CHI
Cleveland Cavaliers CLE
Dallas Mavericks DAL
Denver Nuggets DEN
Detroit Pistons DET
Golden State Warriors GSW
Houston Rockets HOU
Indiana Pacers IND
Los Angeles Clippers LAC
Los Angeles Lakers LAL
Memphis Grizzlies MEM
Miami Heat MIA
Milwaukee Bucks MIL
Minnesota Timberwolves MIN
New Orleans Pelicans NOP
New York Knicks NYK
Oklahoma City Thunder OKC
Orlando Magic ORL
Philadelphia 76ers PHI
Phoenix Suns PHO
Portland Trail Blazers POR
Sacramento Kings SAC
San Antonio Spurs SAS
Toronto Raptors TOR
Utah Jazz UTA
Washington Wizards WAS


,NUMBER,PLAYER,POS,HEIGHT,WEIGHT,BIRTH_DATE,NATIONALITY,EXPERIENCE,COLLEGE,team
0,11,Trae Young,PG,6-1,164.0,1998-09-19,US US,6,Oklahoma,ATL
1,5,Dyson Daniels,SG,6-8,199.0,2003-03-17,AU AU,2,NaN,ATL
2,10,Zaccharie Risacher,SF,6-8,200.0,2005-04-08,ES ES,R,NaN,ATL
3,17,Onyeka Okongwu,C,6-8,235.0,2000-12-11,US US,4,USC,ATL
4,27,Vit Krejci,PG,6-8,195.0,2000-06-19,CZ CZ,3,NaN,ATL


In [ ]:
import pandas as pd
#usunac kolumny!!!!!!

def get_team_per_game_stats(team_abbr: str, year: int):
    url = f"https://www.basketball-reference.com/teams/{team_abbr}/{year}.html"
    tables = pd.read_html(url)

    # Szukamy tabeli "Per Game" – zwykle zawiera kolumny: "Player", "G", "PTS", itp.
    for table in tables:
        if "Player" in table.columns and "G" in table.columns and "PTS" in table.columns:
            table["Team"] = team_abbr  # dodajemy kolumnę z nazwą drużyny
            return table
    return None

all_stats = []

for team_name, team_abbr in nba_teams.items():
    try:
        df = get_team_per_game_stats(team_abbr, 2025)
        if df is not None:
            all_stats.append(df)
    except Exception as e:
        print(f"Nie można pobrać danych dla {team_name} ({team_abbr}): {e}")
combined_stats = pd.concat(all_stats, ignore_index=True)

print(combined_stats.head())

Nie można pobrać danych dla Atlanta Hawks (ATL): HTTP Error 429: Too Many Requests
Nie można pobrać danych dla Boston Celtics (BOS): HTTP Error 429: Too Many Requests
Nie można pobrać danych dla Brooklyn Nets (BRK): HTTP Error 429: Too Many Requests
Nie można pobrać danych dla Charlotte Hornets (CHO): HTTP Error 429: Too Many Requests
Nie można pobrać danych dla Chicago Bulls (CHI): HTTP Error 429: Too Many Requests
Nie można pobrać danych dla Cleveland Cavaliers (CLE): HTTP Error 429: Too Many Requests
Nie można pobrać danych dla Dallas Mavericks (DAL): HTTP Error 429: Too Many Requests
Nie można pobrać danych dla Denver Nuggets (DEN): HTTP Error 429: Too Many Requests
Nie można pobrać danych dla Detroit Pistons (DET): HTTP Error 429: Too Many Requests
Nie można pobrać danych dla Golden State Warriors (GSW): HTTP Error 429: Too Many Requests
Nie można pobrać danych dla Houston Rockets (HOU): HTTP Error 429: Too Many Requests
Nie można pobrać danych dla Indiana Pacers (IND): HTTP Error

KeyboardInterrupt: 

In [ ]:
season_team_stats = get_team_ratings(2025, team=[team for team in nba_teams.values()])
season_team_stats.to_csv(r"C:\Users\tomai\Downloads\team_ratings.csv", index=False)

Retrying after 3169 sec...


In [3]:
get_team_stats(['GSW','BOS'], 2025, data_format='PER_GAME')

Error obtaining data table.


ConnectionError: Request to basketball reference failed